In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import datetime
import json
from pathlib import Path
from matplotlib import cm
import napari

sys.path.append("../complete_pipeline")
from utils import annotate_cropping_windows, crop_all_views, read_first_frame

In [3]:
transform_filters = {
    "mirror-top": "transpose=2,transpose=2,hflip",
    "mirror-bottom": "hflip",
    "mirror-left": "transpose=2,hflip",
    "mirror-right": "transpose=1,hflip",
    "central": "",  # No transformation
}


def get_coordinates(frame, name: str, coordinates):
    viewer = napari.Viewer()
    cmap = cm.get_cmap("hsv", len(coordinates[name]))
    colors = [cmap(i) for i in range(len(coordinates[name]))]

    viewer.add_image(frame, name=name, contrast_limits=[0, 255])
    points_layer = viewer.add_points(
        coordinates[name],
        size=10,
        face_color=colors,
        name="points",
        edge_color="white",
        edge_width=0.5,
    )
    points_layer.editable = True
    napari.run()
    adjusted_points = points_layer.data
    return adjusted_points

def hflip(arr, value):
    points_flipped = np.zeros_like(arr)  # initialize the array
    points_flipped[:, 1] = (
        np.asanyarray(value)[2] - arr[:, 1]
    )  # the new x is the width - the old x
    points_flipped[:, 0] = arr[:, 0]  # the y remains the same
    return points_flipped, np.asanyarray(value)


def transpose1(arr, value):
    points_transposed = np.zeros_like(arr)  # initialize the array
    points_transposed[:, 1] = (
        np.asanyarray(value)[3] - arr[:, 0]
    )  # the new x is the height - the old y
    points_transposed[:, 0] = arr[:, 1]  # the y remains the same
    value_tranps = np.array(
        [value[0], value[1], value[3], value[2]]
    )  # rotate the rectangle to follow transformation
    return points_transposed, value_tranps


def transpose2(arr, value):

    points_transposed = np.zeros_like(arr)  # initialize the array
    points_transposed[:, 1] = arr[:, 0]  # new x is the old y
    points_transposed[:, 0] = value[2] - arr[:, 1]  # new y is the width - the old x
    value_tranps = np.array(
        [value[0], value[1], value[3], value[2]]
    )  # change width and height to reflect the transformation
    return points_transposed, value_tranps


def no_transformation(arr, value):
    return arr, value


def transformation(key, coordinates_cropped, value):
    transform_filters = {
        "mirror-top": [transpose2, transpose2, hflip],
        "mirror-bottom": [hflip],
        "mirror-left": [transpose2, hflip],
        "mirror-right": [transpose1, hflip],
        "central": [no_transformation],
    }
    for f in transform_filters[key]:
        coordinates_cropped, value = f(coordinates_cropped, value)
    return coordinates_cropped

In [4]:
def get_coordinates_arena_and_transform(rectangles, frame):
    """
    Function that allows you to adjust bounding boxes of the arena and get the coordinates for improve calibration

    -args: rectangles for cropping, frame to display the bounding boxes
    -returns: dictionary with the coordinates of the bounding boxes fo the arena transforrmed accordingly to the transformation applied to that point of view

    original coordinates are:
    """

    coordinates = np.load(
        r"D:\P05_3DRIG_YE-LP\e01_mouse_hunting\v04_mice-hunting\20240803\M4\140337\right_coords.pkl",
        allow_pickle=True,
    )

    coordinates_arena = {}

    for key, value in rectangles.items():
        coordinates_arena[key] = np.asanyarray(
            get_coordinates(frame, str(key), coordinates)
        )
        # cropping
        y, x, h, w = value  # try it first

        coordinates_arena[key][:, 1] = coordinates_arena[key][:, 1] - x
        coordinates_arena[key][:, 0] = coordinates_arena[key][:, 0] - y
        print(f"print cropped coordinates for {key} \n {coordinates_arena[key]}")

        # transform them
        coordinates_arena[key] = transformation(key, coordinates_arena[key], value)
    return coordinates_arena

In [ ]:
input_file = r"D:\P05_3DRIG_YE-LP\e01_mouse_hunting\v04_mice-hunting\test_cropping\calibration_retest\Calibration\multicam_video_2024-08-03T16_36_58.avi"
input_file = Path(input_file)

tstamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

frame = read_first_frame(input_file)
rectangles = annotate_cropping_windows(frame)

# insert function to 
coordinates_transfrmed = get_coordinates_arena_and_transform(rectangles, frame)


test_output_dir = input_file.parent / f"test-output_{tstamp}"

cropping_specs = []
